In [73]:
from itertools import combinations
from typing import Callable
from typing import Type
from typing import Tuple
#from collections.abc import Iterable 
#@abstractmethod
#    @property
#    @abstractmethod
# no need to call super
from icecream import ic
import graphviz
    
    
from abc import ABC, abstractmethod
import numpy as np
        
class function(ABC):
    @property
    @abstractmethod
    def forward_func(self) -> Callable:
        pass
    
    @property
    @abstractmethod
    def backward_func(self) -> Callable:
        pass
    
    def get_functions(self) -> Tuple[Callable, Callable]:
        return self.forward_func, self.backward_func
    

class sin(function):
    forward_func = np.sin
    backward_func = np.cos

class tanh(function):
    forward_func = np.tanh
    backward_func = lambda x: 1 - np.square(tanh(x))

class cos(function):
    forward_func = np.cos
    backward_func = lambda x: -np.sin(x)
          

class multiply(function):
    
    forward_func = lambda: None # to be overwitten in constructor
    backward_func = lambda: None # to be overwitten in constructor
    
    def __init__(self,*, allow_arbitrary_many = False): # * makes allow_arbitary_many to keword only
        if not allow_arbitrary_many: # simple case two factors
            self.forward_func = np.multiply
            self.backward_func = lambda x, y: (y, x)
        else:
            forward_func = lambda *x: np.prod(np.vstack(x), axis=0)
            self.backward_func = lambda *x: (np.prod(np.vstack(values), axis=0) for values in combinations(x, len(x) - 1))
                   
            
        
class add(function):
    
    forward_func = lambda: None # to be overwitten in constructor
    backward_func = lambda: None # to be overwitten in constructor
    
    def __init__(self,*, allow_arbitrary_many = False): # new is clalled before the constructer (before: "__init__")
        if not allow_arbitrary_many:
            self.forward_func = np.add 
            self.backward_func = lambda x, y: (np.full_like(x, 1), np.full_like(y, 1)) #arrays with same shape of x and y, filled with 1
        else:
            self.forward_func = lambda *x: np.sum(np.vstack(x), axis=0)
            self.backward_func = lambda *x: (np.full_like(x[0], 1) for _ in range(len(x))) #backward_func = lambda *x: (np.ones(len(x[0]) if isinstance(x[0], Iterable) else 1) for _ in range(len(x))))

            
            
        
    
     
        
    
#b = function2(np.sin, np.cos)
c = sin()
print(c, c.forward_func, c.backward_func)
f1, f2 = c.get_functions()
print(f1, f2)
#c.eval_forward(2)
d = [sin(), sin()]


e = multiply()
print(e.forward_func)


e = multiply(allow_arbitrary_many=True)
print(e.forward_func)
e = multiply(allow_arbitrary_many=False)
print(e.forward_func)
f = cos()
print(f.forward_func(2), np.cos(2))  
forw, backw = c.get_functions()
print(forw, backw) 
ic.disable()
ghj = ic(f.forward_func(2))
ic(ghj)
k = tanh()
l = cos()
ic.enable()
ic(l.forward_func(2))
ic()

        

ic| l.forward_func(2): -0.4161468365471424
ic| 2835660695.py:108 in <module> at 22:56:51.820


<__main__.sin object at 0x7f713a97b0e0> <ufunc 'sin'> <ufunc 'cos'>
<ufunc 'sin'> <ufunc 'cos'>
<ufunc 'multiply'>
<bound method multiply.<lambda> of <__main__.multiply object at 0x7f713a9bf250>>
<ufunc 'multiply'>
-0.4161468365471424 -0.4161468365471424
<ufunc 'sin'> <ufunc 'cos'>


In [74]:
from collections.abc import Sequence
from __future__ import annotations
from numbers import Number

class expr_end_node():
    def __init__(self, value: Number | np.ndarray , grad_value: Number | np.ndarray  = 0):
        self.value = value
        self.grad_value = grad_value
        
class expr_node():
    def __init__(self, func: function, childs: Sequence[expr_node | expr_end_node] = []):
        #self.parants = parants #expr_node
        self.childs = childs  #expr_node
        self.func = func #function 
        # lieber so:
        # self.forward_func, self.backward_func = func.get_functions()
        # und dann self.func entfernen
        
f = sin()
g = sin()
ex1 = expr_node(f)
ex2 = expr_node(g)
ex3 = expr_node(g)
ex1.childs = [ex2]
#ex2.parants = [ex1]
ex2.childs = [ex3]
#sin(sin(sin()))

In [75]:
ex1 = expr_node(sin())
ex2 = expr_node(sin())
ex3 = expr_node(sin())
x = expr_end_node(np.float64(0.345))
ex1.childs = [ex2]
ex2.childs = [ex3]
ex3.childs = [x]

#sin(sin(sin(x)))
"""
sin
 |
sin
 |
sin
 |
 x
"""
#cos(a)*sin(b + tanh(c))
"""
    multiply
    |      |
   cos    sin
   |       |
   a      add
         |  |
         b tanh
             |
             c
"""

#f(g(h(x)))' = f'(g(h(x))) * (g(h(x)))' =  f'(g(h(x))) * g'(h(x)) * h'(x)

# h'(x) * g'(h(x)) * f'(g(h(x)))



#print(x)

'\n    multiply\n    |      |\n   cos    sin\n   |       |\n   a      add\n         |  |\n         b tanh\n             |\n             c\n'

In [76]:
def forward(node):
    return node.func.forward_func(*(forward(child) for child in node.childs)) if type(node) is not expr_end_node else node.value 
forward(ex1)
"""
def forward(node):
    if type(node) is not expr_end_node:
        node.func.forward_func(*(forward(child) for child in node.childs))
    else:
        node.value 
"""
forward(ex1)

def backward(node, value = 1):
    if type(node) is not expr_end_node:
        child_values = (forward(child) for child in node.childs)
        if len(node.childs) == 1:
            backward(node.childs[0], value * node.func.backward_func(*child_values))
        else:
            for child, new_value in zip(node.childs, node.func.backward_func(*child_values), strict=True):
                backward(child, value * new_value)
    else: 
        node.grad_value += value
        
        
#result,target
    
        
        
x.grad_value=0

ic(forward(ex1))
ic(np.sin(np.sin(np.sin(np.float64(.345)))))

backward(ex1)
ic(x.grad_value)
ic(np.cos(np.sin(np.sin(.345)))*np.cos(np.sin(.345))*np.cos(0.345))

print(x.grad_value)


"""
def print_all(a,b,c):
    print(a)
    print(b)
    print(c)

print_all(*[1,2,3])
"""
#zip((a1,a2.a3),(b1,b2,b3) -> ((a1,b1),(a2,b2),(a3,b3))


ic| forward(ex1): 0.32573257710464837
ic| np.sin(np.sin(np.sin(np.float64(.345)))): 0.32573257710464837
ic| x.grad_value: 0.8393506506551831
ic| np.cos(np.sin(np.sin(.345)))*np.cos(np.sin(.345))*np.cos(0.345): 0.8393506506551831


0.8393506506551831


'\ndef print_all(a,b,c):\n    print(a)\n    print(b)\n    print(c)\n\nprint_all(*[1,2,3])\n'

In [77]:
from sklearn.datasets import load_iris
from numpy import linalg as LA



iris = load_iris()
print(len(iris.data), len(iris.target))
print(type(iris.data), type(iris.target))
ic(iris.target)
normalized_data = iris.data/LA.norm(iris.data, axis=0)

"""
out = vector len 3 , out_1: suit 1,  out_1: suit 2, 
suit_1 = (1,0,0)
suit_2 = (0,1,0)
suit_3 = (0,0,1)

netz = 
    in: vecor len 4 
    out = vector len 3
    
with  2-5 hidden layer; 4- 10 neurons each
"""

ic| iris.target: array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
                        0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
                        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
                        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
                        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])


150 150
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


'\nout = vector len 3 , out_1: suit 1,  out_1: suit 2, \nsuit_1 = (1,0,0)\nsuit_2 = (0,1,0)\nsuit_3 = (0,0,1)\n\nnetz = \n    in: vecor len 4 \n    out = vector len 3\n    \nwith  2-5 hidden layer; 4- 10 neurons each\n'

In [78]:
test_prediction = np.array([0.2, 0.6, 0.3])
test_target = np.array([0, 1, 0])

# Mean squared error (MSE):
def loss_mse(prediction, target):
    return np.square(prediction - target).sum()
#(pred_x, pred_y), (target_x, target_y) -> (pred_x - target_x)²+(pred_y - target_y)²



loss1 = ic(loss_mse(test_prediction, test_target))
ic(np.log(test_prediction))
#loss2 = ic(loss_ce(test_prediction,test_target))

#ic.disable()
#ic.enable()

# 0.2²+0.4²+0.3² = 
#ic(loss1(test_prediction, test_target))

ic| loss_mse(test_prediction, test_target): 0.29000000000000004
ic| np.log(test_prediction): array([-1.60943791, -0.51082562, -1.2039728 ])


array([-1.60943791, -0.51082562, -1.2039728 ])

In [79]:
'''
in_1     out_1
in_2     out_2

initial = w, b are random values
netz(in) = 
    out_1 = in_1 * w_{1,1} + in_2 *w_{1,2} + b_1
    out_2 = in_1 * w_{2,1} + in_2 *w_{2,2} + b_2

# net(in, {w}, {b}) 
out = netz(in)
loss(out, true_val)
backpropergation -> change  w, b <--------
'''

'\nin_1     out_1\nin_2     out_2\n\ninitial = w, b are random values\nnetz(in) = \n    out_1 = in_1 * w_{1,1} + in_2 *w_{1,2} + b_1\n    out_2 = in_1 * w_{2,1} + in_2 *w_{2,2} + b_2\n\n# net(in, {w}, {b}) \nout = netz(in)\nloss(out, true_val)\nbackpropergation -> change  w, b <--------\n'

In [107]:
dot = graphviz.Digraph('round-table', comment ='The Round Table')
dot.node('A', 'King Arthur')
dot.node('B', 'Sir Bedevere the Wise')
dot.node('L', 'Sir Lancelot the Brave')
dot.edges(['AB', 'AL'])
dot.edge('B', 'L', constraint='false')

print(dot.source)

dot.render('doctest-output/round-table.gv').replace('\\', '/')
'doctest-output/round-table.gv.pdf'


def print_graph(node, dot, parent_id=""):
    node_identifier = str(id(node))
    if type(node) is not expr_end_node:
        print(node.func.forward_func)
        dot.node(node_identifier, type(node.func).__name__)
        for child in node.childs:
            print_graph(child, dot, node_identifier)
    else:
        print(node.value)
    if parent_id != "":
        dot.edge(parent_id , node_identifier, constraint='false')

dot = graphviz.Digraph('graph', comment='test')
dot.render('doctest-output/func.gv').replace('\\', '/')
'doctest-output/func.gv.pdf'
print_graph(ex1, dot)

// The Round Table
digraph "round-table" {
	A [label="King Arthur"]
	B [label="Sir Bedevere the Wise"]
	L [label="Sir Lancelot the Brave"]
	A -> B
	A -> L
	B -> L [constraint=false]
}

<ufunc 'sin'>
<ufunc 'sin'>
<ufunc 'sin'>
0.345
